## 6. Verificando se há hiatos de tempo em 'time' e preenchendo-os

In [ ]:
def preencher_hiatos(df, col_time='time', col_val='value', hiato=0.26, x='15min', st=15, e=15):
    """
    Preenche hiatos (lacunas) maiores ou iguais a hiato no DataFrame,
    inserindo linhas com timestamps intermediários e valores NaN.

    Parâmetros:
    -----------
    df : pd.DataFrame
        DataFrame original contendo ao menos as colunas de tempo e valor.
    col_time : str, default 'time'
        Nome da coluna com timestamps (deve ser datetime).
    col_val : str, default 'value'
        Nome da coluna com os valores (numéricos, pode conter NaNs).
    hiato : float, default 3
        Mínimo intervalo em horas para considerar um hiato para preenchimento.
    x : str, default '15min'
        Frequência para preenchimento dos timestamps intermediários.
    st : float, default 15
        Determina após quantos minutos do valor antes do hiato iremos colocar nosso primeiro preenchimento
    e : float, default 15
        Determina a distância mínima que o último preenchimento pode estar do primeiro timestamp após o hiato
    Retorna:
    --------
    pd.DataFrame
        Novo DataFrame com os hiatos preenchidos com linhas contendo NaN em col_val,
        ordenado por tempo e com índice resetado.
    """

    df = df.copy()
    
    # criando uma coluna que mede a diferença em horas entre um elemento de 'col_time' e o elemento 'anterior'
    df['diff_horas'] = (df[col_time].diff().dt.total_seconds())/(60 * 60)

    # preencheremos somente hiatos maiores ou iguais a {hiato}

    # índices dos elementos que estão imediatamente antes de cada hiato
    inds_i = df[df['diff_horas'] >= hiato].index - 1
    # índices dos elementos que estão ao fim de cada hiato
    inds_f = df[df['diff_horas'] >= hiato].index

    # Número de hiatos:
    print(f'O total de hiatos encontrados de pelo menos {hiato} horas, foi:', len(inds_i), f'\nIremos preenchê-los com mais marcações, de {x} em {x}.\n')

    # Novas linhas, com respectivos datatimes, que iremos adicionar ao df após o processo.
    # Começamos com uma lista vazia.
    linhas = []

    # Pegaremos cada índice de início e fim de hiato, acessaremos o elemento correspondente em 'col_time' e
    # faremos o preenchimento desse hiato

    for i in range(len(inds_i)):
        inicio = inds_i[i] # índice início hiato (rodaremos a lista de índices)
        fim = inds_f[i] # índice fim hiato

        start_time = df.loc[inicio, col_time] # acessando df na coluna 'col_time' na posição início
        end_time = df.loc[fim, col_time] # na posição fim

        # Lista com os timestamps criados
        range_time = pd.date_range(start=start_time + pd.Timedelta(minutes=st), # começaremos o preenchimento 15 minutos após início do hiato
                                            end=end_time - pd.Timedelta(minutes=e), # terminaremos no máximo até 15 minutos antes do fim do hiato
                                            freq=x) # preencheremos de 15 em 15 minutos esse hiato
        
        # Colocaremos cada um desses timestamps dentro do dicionário com as chaves 'time'
        # e 'value', essas últimas recebendo NaN como valores. O objeto 'linhas' receberá esse dicionário.
        for timestamp in range_time:
            linhas.append({col_time: timestamp, col_val: np.nan}) 


    # O dicionário 'linhas' se torna o dataframe 'df_linhas'
    df_linhas = pd.DataFrame(linhas)

    # Concatenamos o dataframe df, mas sem a coluna diff_horas (agora desnecessária), com
    # o dataframe df_linhas, que tem as mesmas colunas (value e time).
    # ignore_index = True evita índices duplicados (exemplo: o índice 2 em ambos os dataframes), 
    # e reseta o índice do dataframe concatenado, para que vá de 0 até o último elemento da lista, de maneira sequencial
    df = pd.concat([df.drop(columns=['diff_horas']), df_linhas], ignore_index=True)

    # Ordenamos o dataframe por 'time' crescente e mais uma vez resetamos os índices (a redundância
    # se deve apenas à uma questão de organização)
    df = df.sort_values(by=col_time).reset_index(drop=True)

    return df

df_tempos = preencher_hiatos(df_raw)

# Esse é o dataframe com os dados originais e novas estampas de tempo, mas sem preencher os valores
# correspondentes a elas

df_tempos

## 7. Atribuindo valores aos novos timestamps criados

In [ ]:
from scipy.stats import truncnorm

# Função para gerar valores de uma normal truncada (com limite inferior e superior)
def amost_norm_tr(mu, sigma, low=0, high=500):
    a, b = (low - mu) / sigma, (high - mu) / sigma
    return truncnorm.rvs(a, b, loc=mu, scale=sigma)

def imp_est_value(df, col_time='time', col_val='value', n=20):
     
    """
    Imputa valores ausentes em uma coluna de medições temporais usando normal truncada.

    A imputação é feita individualmente para cada timestamp com valor ausente,
    baseado na média e desvio padrão calculados a partir das n vizinhas observações
    mais próximas no tempo (n/2 anteriores e n/2 posteriores), que:
      - possuem o mesmo valor da hora (0 a 23) do dado ausente,
      - possuem valores observados (não NaN),
      - e são distintas do próprio dado ausente.

    Para garantir independência estatística e evitar viés, a média e desvio padrão
    são sempre calculados apenas com os dados originais observados (não imputados),
    mesmo durante o processo de múltiplas imputações.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo as colunas de tempo e valores a imputar.

    col_time : str, default 'time'
        Nome da coluna no DataFrame que contém as datas e horários completos (formato datetime).

    col_val : str, default 'value'
        Nome da coluna contendo os valores numéricos onde podem existir dados ausentes (NaN).

    n : int, default 20
        Número total de vizinhos temporais a considerar para cálculo de média e desvio padrão.
        A função busca n//2 observações anteriores e n//2 posteriores mais próximas no tempo.

    Retorna:
    --------
    pandas.DataFrame
        Cópia do DataFrame original com os valores ausentes imputados.
    """

    df = df.copy()

    # Certificar que col_time está no formato datetime
    df[col_time] = pd.to_datetime(df[col_time])

    # criar a coluna _hora
    df['_hora'] = df[col_time].dt.hour

    # garante ordenação pelo tempo
    df.sort_values(by=col_time, inplace=True)

    # Base fixa só com dados originais (não imputados)
    df_original = df[df[col_val].notna()].copy()

    # Diferenças entre valores e tempos consecutivos
    df_original['diff'] = df_original[col_val].diff()
    df_original['delta'] = df_original[col_time].diff()

    mask_15min = df_original['delta'] == pd.Timedelta(minutes=15)
    diffs_15min = df_original.loc[mask_15min, "diff"].abs()
    mean_diff = diffs_15min.mean()
    q1 = diffs_15min.quantile(0.25)
    q3 = diffs_15min.quantile(0.75)
    intervalo_iqr = (q1,q3)

    # coluna que vai dizer se o valor foi estimado ou não
    df['is_imputed'] = False

    # indíces de elementos de col_val com valor NaN
    inds_nan = df[df[col_val].isna()].index
    
    # Número de medições reais à esquerda e à direita
    metade = n // 2

    for ind in inds_nan:
        timestamp_faltante = df.at[ind, col_time] # selecionar o timestamp sem valor
        hora_faltante = df.at[ind, '_hora'] # selecionar a hora do timestamp sem valor

        # Filtrar dados com a mesma hora e valores não ausentes
        df_valid = df_original[df_original['_hora'] == hora_faltante]

        # Vizinhos anteriores (com timestamp < faltante), pega os mais próximos
        anteriores = df_valid[df_valid[col_time] < timestamp_faltante].tail(metade)

        # Vizinhos posteriores (com timestamp > faltante), pega os mais próximos
        posteriores = df_valid[df_valid[col_time] > timestamp_faltante].head(metade)

        # Junta os vizinhos do dado faltante
        amostras = pd.concat([anteriores, posteriores])

        if len(amostras) >= 2:
            # Distância em segundos até o timestamp ausente
            amostras['delta'] = (amostras[col_time] - timestamp_faltante).dt.total_seconds().abs()

            # Peso inversamente proporcional ao quadrado da distância (suavização forte)
            amostras['peso'] = 1 / (1 + amostras['delta'])**2

            # Extrai valores e pesos
            valores = amostras[col_val].values
            pesos = amostras['peso'].values

            # Média ponderada
            mu_ponderado = np.average(valores, weights=pesos)

            # Variância e desvio padrão ponderados
            var_ponderada = np.average((valores - mu_ponderado) ** 2, weights=pesos)
            sigma_ponderado = np.sqrt(var_ponderada)
            
            # parâmetros mínimo e máximo
            a = df.at[ind - 1, col_val] - (mean_diff + intervalo_iqr[0] + 10)
            b = df.at[ind - 1, col_val] + (mean_diff + intervalo_iqr[1] + 10)

            # Imputa o valor usando normal truncada
            # Mesmo que o valor anterior tenha sido imputado, ele é usado como base
            # para limitar o próximo sorteio, garantindo continuidade e evitando saltos irreais
            imputado = round(amost_norm_tr(mu_ponderado, sigma_ponderado, a, b), 1)
            df.at[ind, col_val] = imputado
            df.at[ind, 'is_imputed'] = True  # Marca como imputado

    # Remove coluna auxiliar
    df.drop(columns=['_hora'], inplace=True)

    return df

df_raw = imp_est_value(df_raw)
df_est = imp_est_value(df_tempos)

# Novo dataframe, com os valores medidos e os valores estimados
df_est